# Sentence-Centric

In [1]:
# Read data

import pandas as pd

frames_filename = "/home/anca/Documents/frames/data/pilot_data_out.csv"
frames_dataset = pd.read_csv(frames_filename)

#frames_dataset.head(n=10)
import sys
stdout = sys.stdout

sys.path.append('../../')

from defaultconfig import Configuration

class FramesConfig(Configuration):
    inputColumns = ["Input.word_phrase", "Input.frames", "Input.sentence", "Input.beg", "Input.end",
                    "Input.vid"]
    outputColumns = ["Answer.FrameType"]
    
    # processing of a closed task
    open_ended_task = False
    annotation_vector = [
        "activity_resume", "adjusting", "adorning", "aggregate", "amalgamation", "amounting_to", "appointing", "arranging",
        "assemble", "assessing", "assistance", "attaching", "attempt_suasion", "becoming", "becoming_aware",
        "behind_the_scenes", "be_in_agreement_on_assessment", "being_attached", "being_employed", "being_in_effect",
        "being_in_operation", "being_located", "being_named", "being_necessary", "besieging", "body_mark",
        "body_movement", "bond_maturation", "bringing", "building", "buildings", "capacity", "categorization",
        "causation", "cause_change", "cause_change_of_phase", "cause_impact", "cause_motion", "cause_to_amalgamate",
        "cause_to_be_included", "cause_to_be_wet", "cause_to_end", "cause_to_fragment", "cause_to_start",
        "change_event_duration", "change_of_leadership", "change_operational_state", "change_position_on_a_scale",
        "change_tool", "closure", "cogitation", "collaboration", "color", "come_together", "coming_to_be",
        "coming_up_with", "commitment", "communicate_categorization", "communication", "commutative_process",
        "compatibility", "conquering", "contacting", "containing", "create_physical_artwork", "cure", "damaging",
        "defend", "deny_permission", "differentiation", "dimension", "distinctiveness", "diversity",
        "dominate_competitor", "dominate_situation", "earnings_and_losses", "eclipse", "education_teaching",
        "emotion_active", "emotion_directed", "employing", "emptying", "encounter", "enforcing", "evaluative_comparison",
        "event", "eventive_affecting", "evidence", "exchange_currency", "exercising", "expensiveness",
        "experiencer_focus", "experiencer_obj", "expertise", "explaining_the_facts", "feeling", "filling", "first_rank",
        "fluidic_motion", "forging", "forgiveness", "gesture", "getting", "gizmo", "have_as_requirement",
        "have_associated", "hindering", "identicality", "impact", "imposing_obligation", "inchoative_attaching",
        "influence_of_event_on_cognizer", "ingredients", "intentionally_affect", "intoxicants", "judgment_communication",
        "labor_product", "leadership", "left_to_do", "locale_by_event", "locale_by_ownership", "make_agreement_on_action",
        "make_possible_to_do", "manipulation", "manufacturing", "meet_specifications", "meet_with_response", "membership",
        "memorization", "motion_directional", "needing", "nuclear_process", "objective_influence", "offering",
        "operating_a_system", "opinion", "part_inner_outer", "part_whole", "path_shape", "people", "people_by_morality",
        "performers_and_roles", "permitting", "placing", "posing_as", "posture", "prevent_from_having", "preventing",
        "processing_materials", "progress", "prohibiting", "protecting", "quantity", "reasoning", "recording",
        "referring_by_name", "regard", "rejuvenation", "remainder", "remembering_experience", "removing", "replacing",
        "reporting", "request", "required_event", "reshaping", "residence", "resolve_problem", "resurrection",
        "reveal_secret", "revenge", "rewards_and_punishments", "rotting", "scrutiny", "similarity",
        "spelling_and_pronouncing", "state_continue", "state_of_entity", "studying", "subjective_influence",
        "subordinates_and_superiors", "subversion", "successful_action", "success_or_failure", "supply", "surpassing",
        "surviving", "temporary_stay", "terms_of_agreement", "text_creation", "thriving", "thwarting", "topic", "travel",
        "type", "undergo_change", "waver_between_options", "working_on", "none"
    ]
    
    def processJudgments(self, judgments):
        # change default separator to whitespace to make it work with our file
        for col in self.outputColumns:
            judgments[col] = judgments[col].apply(lambda x: str(x).replace('|',','))
            
            judgments[col] = judgments[col].apply(lambda x: str(x).replace('None of the above.','none'))
            judgments[col] = judgments[col].apply(lambda x: str(x).replace(' ','_'))
            judgments[col] = judgments[col].apply(lambda x: str(x).lower())
        return judgments

config = FramesConfig()

# pre-process the data and create the annotation vectors
from controllers.inputController import processFile
pre_processed_results = processFile(
    root=".", directory="", filename=frames_filename,
    config=config
)

In [2]:
# %%debug

sys.stdout = stdout

# run the metrics
from models import Metrics
processed_results = Metrics.run(pre_processed_results, config)

In [3]:
import operator

dlist = []
max_frame = []
max_fss = []
for idx in processed_results["units"].index:
    if idx != "unit_annotation_score":
        aux_dict = { k:v for k, v in processed_results["units"]["unit_annotation_score"][idx].iteritems() if v }
        dlist.append(aux_dict)
        max_frame.append(max(aux_dict.iteritems(), key=operator.itemgetter(1))[0])
        max_fss.append(max(aux_dict.values()))
#dlist.append({})
#max_frame.append("")
#max_fss.append(0)

processed_results["units"]["FSS"] = pd.Series(dlist, index=processed_results["units"].index)
processed_results["units"]["max_frame"] = pd.Series(max_frame, index=processed_results["units"].index)
processed_results["units"]["max_fss"] = pd.Series(max_fss, index=processed_results["units"].index)

processed_results["units"].to_csv("pilot_data_out_sentence_metrics.csv")

In [4]:
processed_results["units"]

,duration,input.beg,input.end,input.frames,input.sentence,input.vid,input.word_phrase,job,metrics.avg_annotations,metrics.avg_cos_clarity,...,output.FrameType,output.FrameType.annotations,output.FrameType.cos_clarity,output.FrameType.unique_annotations,worker,uqs,unit_annotation_score,FSS,max_frame,max_fss
unit,,,,,,,,,,,,,,,,,,,,,
307L9TDWJY49TADR5GP2JGU75PBN3A,179.066667,0,8,"f:Cure,f:Manipulation,f:People,f:Expertise,f:I...",Kneading promotes the formation of gluten stra...,VID-302541,kneading,/home/anca/Documents/frames/data/pilot_data_out,18.0,0.953206,...,"{u'judgment_communication': 0, u'similarity': ...",18,0.953206,3,15,0.763137,"{u'judgment_communication': 0.0, u'similarity'...","{u'impact': 0.260903462453, u'none': 0.0509407...",manipulation,0.901392
30EMX9PEVKVC5LD3YIYNZWLF3A5KSU,356.333333,107,119,"f:Building,f:Activity_resume,f:Coming_up_with,...","In the late 1980s, with the advent of cognitiv...",VID-066190,rehabilitate,/home/anca/Documents/frames/data/pilot_data_out,22.0,0.872503,...,"{u'judgment_communication': 0, u'similarity': ...",22,0.872503,3,15,0.805904,"{u'judgment_communication': 0.0, u'similarity'...","{u'building': 0.140943654371, u'cure': 0.43926...",rejuvenation,0.938539
30IRMPJWDZV046DQJTYI9MGJH8MKR6,473.642857,53,60,"f:Cure,f:Manipulation,f:People,f:Expertise,f:I...","In industrial production, a slurry of wheat fl...",VID-302522,kneaded,/home/anca/Documents/frames/data/pilot_data_out,17.0,0.976546,...,"{u'judgment_communication': 0, u'similarity': ...",17,0.976546,5,14,0.792973,"{u'judgment_communication': 0.0, u'similarity'...","{u'impact': 0.10608742649, u'expertise': 0.046...",manipulation,0.893913
30QQTY5GMKW4PJYN4T4EASYHINQ7UR,623.266667,116,126,"f:Cure,f:Gizmo,f:Supply,f:People","William Knight, the King's secretary, was sent...",VID-155118,dispensing,/home/anca/Documents/frames/data/pilot_data_out,16.0,0.950255,...,"{u'judgment_communication': 0, u'similarity': ...",16,0.950255,3,15,0.646351,"{u'judgment_communication': 0.0, u'similarity'...","{u'none': 0.157211800072, u'gizmo': 0.12879620...",supply,0.804112
30UZJB2POHO53L7L1GTGPZULE4B350,406.866667,241,247,"f:Undergo_change,f:Cause_change,f:Becoming",The following year the United Kingdom announce...,VID-069871,become,/home/anca/Documents/frames/data/pilot_data_out,23.0,0.866578,...,"{u'judgment_communication': 0, u'similarity': ...",23,0.866578,3,15,0.791787,"{u'judgment_communication': 0.0, u'similarity'...","{u'cause_change': 0.0403411005824, u'becoming'...",becoming,0.924209
30ZKOOGW2WI6ZO5N986E9HVA8YYA1L,660.266667,121,133,"f:Progress,f:Cure,f:Undergo_change","On 23 December 1788, Thompson seriously fractu...",VID-180990,convalescing,/home/anca/Documents/frames/data/pilot_data_out,27.0,0.597351,...,"{u'judgment_communication': 0, u'similarity': ...",27,0.597351,4,15,0.498116,"{u'judgment_communication': 0.0, u'similarity'...","{u'progress': 0.614182362901, u'none': 0.05314...",progress,0.614182
31ANT7FQN8EKXTEKG1IITTVVX9T5H9,597.666667,153,159,"f:Cogitation,f:Objective_influence,f:Causation...",One may distinguish between this positional ''...,VID-030501,varied,/home/anca/Documents/frames/data/pilot_data_out,18.0,0.975426,...,"{u'judgment_communication': 0, u'similarity': ...",18,0.975426,3,15,0.817899,"{u'judgment_communication': 0.0, u'similarity'...","{u'objective_influence': 0.244073813218, u'div...",diversity,0.925910
31D0ZWOD0AB6SZK5DP2EVCC036SA0I,440.066667,233,239,"f:Fluidic_motion,f:Cure,f:Ingredients,f:Removi...","Ovid, also makes Cerberus the cause of the poi...",VID-148474,spewed,/home/anca/Documents/frames/data/pilot_data_out,17.0,0.987484,...,"{u'judgment_communication': 0, u'similarity': ...",17,0.987484,3,15,0.829652,"{u'judgment_communication': 0.0, u'similarity'...","{u'removing': 0.107864502689, u'none': 0.05906...",fluidic_motion,0.940934
32CXT5U14GF8PX36EYWKOSUCT8W8UH,485.800000,106,112,"f:Defend,f:Dimension,f:Change_event_duration,f...",For then:\nOnce the Euclidean plane has been d...,VID-221540,extend,/home/a

In [5]:
unique_sentences = processed_results["units"]["input.sentence"]
unique_frames = config.annotation_vector

sentences = []
frames = []
words = []
sent_text = []
fss = []

for idx in processed_results["units"].index:
    sent_frames = processed_results["units"]["input.frames"][idx].split(",")
    sent_frames = [x.replace('f:','').lower() for x in sent_frames]
    for frame in sent_frames:
        sentences.append(processed_results["units"]["input.vid"][idx])
        words.append(processed_results["units"]["input.word_phrase"][idx])
        sent_text.append(processed_results["units"]["input.sentence"][idx])
        frames.append(frame)
        if frame in processed_results["units"]["FSS"][idx].keys(): 
            fss.append(processed_results["units"]["FSS"][idx][frame])
        else:
            fss.append(0.0)
    #frame = "none"
    #sentences.append(processed_results["units"]["input.vid"][idx])
    #words.append(processed_results["units"]["input.word_phrase"][idx])
    #sent_text.append(processed_results["units"]["input.sentence"][idx])
    #frames.append(frame)
    #if frame in processed_results["units"]["FSS"][idx].keys(): 
    #    fss.append(processed_results["units"]["FSS"][idx][frame])
    #else:
    #    fss.append(0.0)

In [6]:
frame_sentence_score = pd.DataFrame({
    "sent_id" : sentences,
    "sentence" : sent_text,
    "word" : words,
    "fss_sent_centric" : fss,
    "frame" : frames
})

#frame_sentence_score.set_index([frame_sentence_score["sent_id"], frame_sentence_score["frame"]])
frame_sentence_score

,frame,fss_sent_centric,sent_id,sentence,word
0,cure,0.000000,VID-302541,Kneading promotes the formation of gluten stra...,kneading
1,manipulation,0.901392,VID-302541,Kneading promotes the formation of gluten stra...,kneading
2,people,0.000000,VID-302541,Kneading promotes the formation of gluten stra...,kneading
3,expertise,0.000000,VID-302541,Kneading promotes the formation of gluten stra...,kneading
4,impact,0.260903,VID-302541,Kneading promotes the formation of gluten stra...,kneading
5,building,0.140944,VID-066190,"In the late 1980s, with the advent of cognitiv...",rehabilitate
6,activity_resume,0.000000,VID-066190,"In the late 1980s, with the advent of cognitiv...",rehabilitate
7,coming_up_with,0.000000,VID-066190,"In the late 1980s, with the advent of cognitiv...",rehabilitate
8,cure,0.439262,VID-066190,"In the late 1980s, with the advent of cognitiv...",rehabilitate
9,rejuvenation,0.938539,VID-066190,"In the late 1980s, with the advent of cognitiv...",rehabilitate


# Frame-Centric

In [7]:
# Read data

import pandas as pd

frames_filename = "/home/anca/Documents/frames/data/crowdsourcing/output/frame-centric/frame-centric.csv"
frames_dataset = pd.read_csv(frames_filename)

#frames_dataset.head(n=10)
import sys
stdout = sys.stdout

sys.path.append('../../')

from defaultconfig import Configuration

class FramesConfig(Configuration):
    inputColumns = [
        'Input.frame', 'Input.definition', 'Input.examples', 'Input.elements', 'Input.sentences', 'Input.sent_ids',
        'Input.words', 'Input.b', 'Input.e', 'Input.nr_sentences']
    outputColumns = ["Answer.PhraseType"]
    
    # processing of a closed task
    open_ended_task = False
    annotation_vector = [
        'VID-085792', 'VID-089603', 'VID-233373', 'VID-031004', 'VID-258632', 'VID-082583', 'VID-126283', 'VID-220213',
        'VID-162078', 'VID-142386', 'VID-192887', 'VID-285371', 'VID-131438', 'VID-302863', 'VID-023310', 'VID-158087',
        'VID-148474', 'VID-102596', 'VID-176379', 'VID-257483', 'VID-254047', 'VID-058292', 'VID-242436', 'VID-202719',
        'VID-255948', 'VID-105766', 'VID-220982', 'VID-087335', 'VID-071412', 'VID-192404', 'VID-130260', 'VID-166098',
        'VID-170613', 'VID-069871', 'VID-083543', 'VID-200845', 'VID-302541', 'VID-214268', 'VID-005978', 'VID-233784',
        'VID-302522', 'VID-046304', 'VID-286940', 'VID-280283', 'VID-115123', 'VID-291057', 'VID-199596', 'VID-269212',
        'VID-210467', 'VID-017574', 'VID-289962', 'VID-178986', 'VID-299317', 'VID-134226', 'VID-243403', 'VID-290871',
        'VID-190240', 'VID-136492', 'VID-129314', 'VID-267848', 'VID-234943', 'VID-236096', 'VID-281729', 'VID-087382',
        'VID-058986', 'VID-155118', 'VID-198695', 'VID-181827', 'VID-201020', 'VID-030501', 'VID-220756', 'VID-230385',
        'VID-258048', 'VID-294891', 'VID-022055', 'VID-294933', 'VID-099209', 'VID-288267', 'VID-252380', 'VID-134640',
        'VID-033766', 'VID-066190', 'VID-144272', 'VID-272517', 'VID-002758', 'VID-041832', 'VID-250559', 'VID-174451',
        'VID-257807', 'VID-233815', 'VID-062346', 'VID-236618', 'VID-221540', 'VID-081390', 'VID-223089', 'VID-180990',
        'VID-120747', 'VID-008567', 'VID-076231', 'VID-058677', 'VID-211742', 'VID-044730', 'none'
    ]
    
    def processJudgments(self, judgments):
        # change default separator to whitespace to make it work with our file
        for col in self.outputColumns:
            judgments[col] = judgments[col].apply(lambda x: str(x).replace('|',','))
            
            judgments[col] = judgments[col].apply(lambda x: str(x).replace('None of the above.','none'))
        return judgments

config = FramesConfig()

# pre-process the data and create the annotation vectors
from controllers.inputController import processFile
pre_processed_results = processFile(
    root=".", directory="", filename=frames_filename,
    config=config
)

/home/anca/Documents/frames/data/crowdsourcing/output/frame-centric/frame-centric.csv: [##########] 100.0% DONE


In [8]:
# %%debug 

sys.stdout = stdout

# run the metrics
from models import Metrics
processed_results = Metrics.run(pre_processed_results, config)

1 iterations; max d= 1.0 ; wqs d= 0.689790105973; sqs d= 0.559256258412; rqs d= 0.55835692912
2 iterations; max d= 0.458897051756 ; wqs d= 0.0796929561961; sqs d= 0.0490019098108; rqs d= 0.0599517823568
3 iterations; max d= 0.0852631998318 ; wqs d= 0.0130232722956; sqs d= 0.0117669665593; rqs d= 0.010739799771
4 iterations; max d= 0.0405051546764 ; wqs d= 0.00592989610766; sqs d= 0.00662023936093; rqs d= 0.00436672356177
5 iterations; max d= 0.0224014976886 ; wqs d= 0.00233821864423; sqs d= 0.00181605801559; rqs d= 0.00114113399635
6 iterations; max d= 0.012219053992 ; wqs d= 0.00113462321486; sqs d= 0.00141794287866; rqs d= 0.000811956063244
7 iterations; max d= 0.00730156510099 ; wqs d= 0.000633584688616; sqs d= 0.000447493857113; rqs d= 0.000281710179991
8 iterations; max d= 0.00410204525059 ; wqs d= 0.000290709104196; sqs d= 0.000360728507579; rqs d= 0.000199350396109
9 iterations; max d= 0.00238146993065 ; wqs d= 0.000182995351662; sqs d= 0.000129460245836; rqs d= 8.43208766761e-0

In [9]:
processed_results["units"]

,duration,input.b,input.definition,input.e,input.elements,input.examples,input.frame,input.nr_sentences,input.sent_ids,input.sentences,...,metrics.avg_annotations,metrics.avg_cos_clarity,metrics.avg_unique_annotations,output.PhraseType,output.PhraseType.annotations,output.PhraseType.cos_clarity,output.PhraseType.unique_annotations,worker,uqs,unit_annotation_score
unit,,,,,,,,,,,,,,,,,,,,,
301KG0KX9CXSYWVXYIGKVMSJGO02HK,1264.40,"13,9",A <em>Communicator</em> communicates a judgmen...,"14,10","Communicator,Evaluee,Addressee",' Gaston <span id='phrase_question'>denounced<...,f:Judgment_communication,2,"VID-272517,VID-288267",'It would be also the first mission to fire th...,...,7.0,0.727607,3.0,"{u'VID-062346': 0, u'VID-033766': 0, u'VID-220...",7,0.727607,3,5,0.366809,"{u'VID-062346': 0.0, u'VID-033766': 0.0, u'VID..."
302OLP89DZJQMRFLP5VYQDE1DI6AC3,83.60,"16,39,31,11,11,6,18,9,13,1",An <em>Agent</em> or <em>Cause</em> causes an ...,"17,40,32,12,12,7,19,10,14,2","Agent,Cause,Entity,Attribute,Initial category,...","'Hence, when (Hg 0 ) is <span id='phrase_que...",f:Cause_change,10,"VID-136492,VID-257807,VID-134226,VID-002758,VI...","'In Christian theology, people are created goo...",...,15.0,0.557086,9.0,"{u'VID-062346': 0, u'VID-033766': 0, u'VID-220...",15,0.557086,9,5,0.289089,"{u'VID-062346': 0.0, u'VID-033766': 0.0, u'VID..."
304QEQWKZPWVU3E6N87WW7W5VCTO09,21.00,"9,2,23,6",Nouns denoting <em>Aggregate</em>s of <em>Indi...,"10,3,24,7","Aggregate,Individuals,AggregateProperty",'An <span id='phrase_question'> army </span> ...,f:Aggregate,4,"VID-220213,VID-201020,VID-030501,VID-181827",'The EEA is responsible for developing the net...,...,8.0,0.534522,5.0,"{u'VID-062346': 0, u'VID-033766': 0, u'VID-220...",8,0.534522,5,4,0.279584,"{u'VID-062346': 0.0, u'VID-033766': 0.0, u'VID..."
304QEQWKZPWVU3E6N87WW7W5VCU0OM,48.80,"27,0,3,37,4,0,12,15,11,18","An <em>Entity</em> changes, either in its cate...","28,1,4,38,5,1,13,16,12,19","Entity,Attribute,Initial category,Final catego...",' At least one relative instruction has <span...,f:Undergo_change,10,"VID-236096,VID-302863,VID-170613,VID-069871,VI...",'The treaty establishing the European Space Ag...,...,17.0,0.493197,8.0,"{u'VID-062346': 0, u'VID-033766': 0, u'VID-220...",17,0.493197,8,5,0.293390,"{u'VID-062346': 0.0, u'VID-033766': 0.0, u'VID..."
306996CF6WWBIYAFFM6A4SFGR9PB1A,187.60,"0,6,5,19",A <em>Recipient</em> starts off without the <e...,"1,7,6,20","Recipient,Theme,Source",' I <span id='phrase_question'>got</span> two ...,f:Getting,4,"VID-302863,VID-144272,VID-022055,VID-099209","'Recuperating at Swarthmoor, he began dictatin...",...,13.0,0.596285,4.0,"{u'VID-062346': 0, u'VID-033766': 0, u'VID-220...",13,0.596285,4,5,0.608491,"{u'VID-062346': 0.0, u'VID-033766': 0.0, u'VID..."
307L9TDWJY49TADR5GP2JGU78FG3NE,297.20,"9,3",An <em>Agent</em> puts a complex <em>Theme</em...,"10,4","Agent,Theme,Configuration","'Try to order them by importance . CNI ','...",f:Arranging,2,"VID-220213,VID-170613",'The EEA is responsible for developing the net...,...,6.0,0.894427,2.0,"{u'VID-062346': 0, u'VID-033766': 0, u'VID-220...",6,0.894427,2,5,0.771871,"{u'VID-062346': 0.0, u'VID-033766': 0.0, u'VID..."
3087LXLJ6MS1OQ1DO5O69DNR3B30F0,193.40,"11,30",An <em>Arguer</em> presents a <em>Content</em>...,"12,31","Arguer,Content,Support,Addressee",'Jan <span id='phrase_question'>proved</span> ...,f:Reasoning,2,"VID-131438,VID-134640",'Salts of Cs are usually colourless unless the...,...,5.0,0.666667,3.0,"{u'VID-062346': 0, u'VID-033766': 0, u'VID-220...",5,0.666667,3,5,0.258150,"{u'VID-062346': 0.0, u'VID-033766': 0.0, u'VID..."
308KJXFUJRI788HWBRQOFNE51AVATN,250.20,"13,31,9,5,1,23",People's knowledge or skill in certain domains...,"14,32,10,6,2,24",NaN,'He is an <span id='phrase_question'>experienc...,f:Expertise,6,"VID-046304,VID-134226,VID-302522,VID-214268,VI...",'Although in 1955 reserpine was shown to be mo...,...,12.0,0.730297,7.0,"{u'VID-062346': 0, u'VID-033766': 0, u'VID-220...",12,0.730297,7,5,0.

In [10]:
import numpy as np

unique_frames = np.unique(processed_results["units"]["input.frame"])
unique_sentences = config.annotation_vector

frames = []
sentences = []
numerator = []
denominator = []
judgments = []

for idx in processed_results["units"].index:
    frame_sent_arr = processed_results["units"]["input.sent_ids"][idx].split(",")
    frame_judg = processed_results["judgments"].loc[processed_results["judgments"]["unit"] == idx]
    
    for sent in frame_sent_arr:
        # print sent + " ; " + processed_results["units"]["input.frame"][idx]
        frames.append(processed_results["units"]["input.frame"][idx].replace('f:', '').lower())
        sentences.append(sent)
        
        num = 0.0
        den = 0.0
        for jdx in frame_judg.index:
            num += (frame_judg["output.PhraseType"][jdx][sent] * 
                    processed_results["workers"]["wqs"][frame_judg["worker"][jdx]])
            den += processed_results["workers"]["wqs"][frame_judg["worker"][jdx]]
        numerator.append(num)
        denominator.append(den)
        judgments.append(len(frame_judg.index))

sent_frame_res = pd.DataFrame({
    "frame" : frames,
    "sent" : sentences,
    "num" : numerator,
    "den" : denominator,
    "judgments" : judgments
})

In [11]:
sent_frame_res

agg_res = sent_frame_res.groupby(["frame", "sent"]).sum()
agg_res

,,den,judgments,num
frame,sent,,,
activity_resume,VID-066190,5.419611,15,2.393435
activity_resume,VID-190240,5.419611,15,4.186425
activity_resume,VID-192887,5.419611,15,4.545512
activity_resume,VID-242436,5.419611,15,5.131858
activity_resume,VID-290871,5.419611,15,4.756799
adjusting,VID-170613,4.292906,13,2.946771
adjusting,VID-220213,4.292906,13,2.754753
adjusting,VID-288267,4.292906,13,2.244290
adorning,VID-017574,5.160585,14,2.140352


In [12]:
#agg_res.loc[("activity_resume", "VID-066190")]["den"]

fss_frame_centric = []
fc_judg = []
for idx in frame_sentence_score.index:
    if (frame_sentence_score["frame"][idx], frame_sentence_score["sent_id"][idx]) in agg_res.index:
        aux = agg_res.loc[(frame_sentence_score["frame"][idx], frame_sentence_score["sent_id"][idx])]
        fss_frame_centric.append(aux["num"] / aux["den"])
        fc_judg.append(aux["judgments"])
    else:
        fss_frame_centric.append(None)
        fc_judg.append(0)
frame_sentence_score["fss_frame_centric"] = fss_frame_centric
frame_sentence_score["frame_centric_judgments"] = fc_judg

In [13]:
frame_sentence_score.to_csv("frame_sentence_score.csv")